In [1]:
import copy
steps = []
def extend_tree(G, k, T, I, B, L, X, recall):
    #record all steps
    recall = 0
    
    steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])
    # Halt - yes
    if len(L) + len(B) >= k:
        return True
    
    # Halt - No
    if len(B) == 0:
        steps[-1][5] += 1
        return False
    
    # Simplication rule：If exist v∈B with N_G(v) ∩ X = 0, then move v to L
    B_to_L = {v for v in B if not set(G.neighbors(v)) & X}
    for v in B_to_L:
        B.remove(v)
        L.add(v)
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])

    #Select u from B
    for u in list(B):
        
        # branch 1: try to move a vertex from B to L
        B.remove(u)
        L.add(u)  
        if extend_tree(G, k, T, I, B, L, X, recall):
            return True
        # recall
        L.remove(u)
        B.add(u)
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])

        # branch 2: handle u as a internal vertex (Branching Lemma)
        original_u = u  # save the original vertex
        added_edges = []  # save added edges
        B.remove(u)
        I.add(u)
        neighbors_X = set(G.neighbors(u)) & X
        
        #Follow Path Lemma
        while len(neighbors_X) == 1:
            v = neighbors_X.pop()
            X.remove(v)
            T.add_edge(u, v)
            added_edges.append((u, v))
            neighbors_X = set(G.neighbors(v)) & X
            u = v  # update v as new u(current vertex)
            if len(neighbors_X) == 1:
                I.add(u)
            elif len(neighbors_X) != 1:
                B.add(u)
            steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])
        
        B.discard(u)
        counter = 1
        total_items = len(neighbors_X)
        for v in neighbors_X:
            T.add_edge(u, v)
            added_edges.append((u, v))
            B.add(v)
            X.remove(v)
            if counter < total_items:
                steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])
            counter += 1
    
        if extend_tree(G, k, T, I, B, L, X, recall):
            return True
    
        # recall
        for u, v in added_edges:
            T.delete_edge((u, v))  # delete added edges
            I.discard(v)
            X.add(v)
        I.discard(original_u)  # remove original u
        B.add(original_u)  # add original u to B
        X.update(neighbors_X)  # update unused vertices to X
        for v in neighbors_X:
            B.discard(v)  # remove vertices from B
        steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])
    steps[-1][5] += 1
    return False

def max_leaf_spanning_tree(G, k):
    T = Graph()
    I = set()
    L = set()
    B = set()
    X = set(G.vertices())
    recall = 0
    # save initial state
    steps.append([copy.deepcopy(T), copy.deepcopy(I), copy.deepcopy(B), copy.deepcopy(L), copy.deepcopy(X), copy.deepcopy(recall)])
    for r in G.vertices():
        T = Graph() #a tree in G
        T.add_vertex(r) #add initial vertex to T
        I = set()  # the internal vertices of T, with r∈I
        L = set()  # the remaining leaves of T
        B = {r}  # a subset of the leaves of T where T may be extended: the boundary set
        X = set(G.vertices()) - {r}  # the external vertices V\V(T)

        if extend_tree(G, k, T, I, B, L, X, recall):
            return True, T
    return False, None


In [2]:
# use example
G = Graph([('a', 'b'), ('b', 'c'), ('c', 'a'), ('c', 'd'), ('d', 'e')])  
k = 3  # target least number of leaves
result, T = max_leaf_spanning_tree(G, k)
if result:
    print("Exist a spanning tree with maximum leaf number of {}.".format(k))
else:
    print("Not Exist a spanning tree with maximum leaf number of {}.".format(k))

Not Exist a spanning tree with maximum leaf number of 4.


In [3]:
@interact
def show_step(step=slider(0, len(steps)-1, 1, label="Step")):
    T, I, B, L, X, recall = steps[step]
    # vertex colors
    vertex_colors = {'green': list(L), 'red': list(B), 'white': list(X), 'yellow': list(I)}

    # edge_colors
    edge_colors = {'yellow': T.edges(labels=False)}
    
    # plot graph
    G_plot = G.plot(vertex_colors=vertex_colors, edge_colors=edge_colors, layout='circular')
    
    # show current steps of algorithm
    
    print("The current tree T are represented in yellow, the nodes in B are in red, the nodes in L are in green,")
    print(" and the remaining unchanged parts(X) set to white.")
    print(f"Current state: I:{I}, B:{B}, L:{L}, X:{X}, k:{k}")
    if recall == 1:
        print(f"The algorithm has hit a dead end, this state will recall {recall} steps.")
    elif recall == 3:
        print(f"This initial vertex is not available, the state will recall to initial graph and choose a new initial vertex.")
    if step == len(steps)-1:
        if result:
            print(f"The number of sum of vertices in B and vertices in L has reached k:{k}.")
            print(f"YES. The branching algorithm find a maximum leaf spanning tree.")
        else:
            print(f"The branching algorithm has test all viable tree, but can't reach k:{k}.")
            print(f"NO. The branching algorithm find out there is no maximum leaf spanning tree in G.")
    show(G_plot)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.
